## Tasks for Barmer Data: Improvements based on feedback first tasks
    [x] Adjust duration by adding one day
    [x] Translate everything into english
    [x] Redo every task with new Duration
    [x] Task 3: Sort the x-axis in ascending order, i.e. from 1 – 2- 3- 4- etc. (I think now it is ordered by frequency
    [x] Task 4: Sort from longest duration (left) to shortest duration (right)
    [x] Task 5: Reverse x and y axis, sort from longest to shortest
    [x] Task 7: Do several graphs similar to Task 6 but with 4-5 industries (4-5 separate lines) per graph, group those industries that have similar patterns

## 1. Dependencies 

In [366]:
import pandas as pd
import numpy as np

## 2. Importing data I will be working on

##### Techniker Data 

In [367]:
tk_duration = pd.read_excel('Techniker/tk_duration.xlsx')

In [368]:
tk_wirtschaftanzeige = pd.read_excel('Techniker/tk_wirtschaftanzeige.xlsx')

In [369]:
tk_duration.head()

,land,kennzahl,dauer,anzahl,jahr,monat
0,AT,Laenge der Entsendung,< 1 monat,1721,2019,1
1,AT,Laenge der Entsendung,1-3 monate,67,2019,1
2,AT,Laenge der Entsendung,3-12 monate,98,2019,1
3,AT,Laenge der Entsendung,12-24 monate,74,2019,1
4,AT,Laenge der Entsendung,> 24 monate,1,2019,1


In [370]:
tk_wirtschaftanzeige.head()

,land,kennzahl,kategorie,anzahl,jahr,monat
0,AT,Anzahl Wirtschaftszweig,1,6,2019,1
1,AT,Anzahl Wirtschaftszweig,2,1,2019,1
2,AT,Anzahl Wirtschaftszweig,3,643,2019,1
3,AT,Anzahl Wirtschaftszweig,4,21,2019,1
4,AT,Anzahl Wirtschaftszweig,5,5,2019,1


##### Barmer Data

In [371]:
barmer = pd.read_excel('Barmer/barmerAddDuration.xlsx')

In [372]:
# barmer.dtypes
barmer.head() # Check head

,Unnamed: 0,Lnd,Datum Beginn,Datum Ende,Wirtschaftssektor,Duration
0,0,FR,2019-07-02,2019-07-04,21.0,2
1,1,AT,2019-06-12,2019-06-12,21.0,0
2,2,IT,2019-06-16,2019-06-20,21.0,4
3,3,PL,2019-07-08,2019-07-11,21.0,3
4,4,BE,2019-06-26,2019-06-27,21.0,1


In [373]:
'''Checking for the 107 missing values'''
barmer['Lnd'].isna().sum() # Indeed, 107 observations with missing destinations.

'''Checking for other columns'''
barmer["Wirtschaftssektor"].isna().sum() # 183 observations with missing Wirtschaftssektor. 
barmer["Datum Beginn"].isna().sum() # 0 observations with missing Datum Beginn.
barmer["Datum Ende"].isna().sum() # 32 observations with missing Datum Ende.
barmer["Duration"].isna().sum() # 0 observations with missing Duration.

(barmer['Duration']=='Missing').sum() # 32 observations with Missing.

'''Filling missing values with string: "missing"'''
barmer['Lnd'].fillna("Missing", inplace=True) # (Corrected missing
barmer["Wirtschaftssektor"].fillna("Missing", inplace=True) # (Corrected missing)
barmer["Datum Ende"].fillna("Missing", inplace=True) # (Corrected missing)
barmer.head()

,Unnamed: 0,Lnd,Datum Beginn,Datum Ende,Wirtschaftssektor,Duration
0,0,FR,2019-07-02,2019-07-04 00:00:00,21,2
1,1,AT,2019-06-12,2019-06-12 00:00:00,21,0
2,2,IT,2019-06-16,2019-06-20 00:00:00,21,4
3,3,PL,2019-07-08,2019-07-11 00:00:00,21,3
4,4,BE,2019-06-26,2019-06-27 00:00:00,21,1


## 3. Adding one day to duration

In [374]:
#barmer['Duration'].dtypes # Returns object type

'''In duration there were Missing values categorized with the string value "Missing". 
Before I tranformed the object data into integer data in order to add 1 day to every observation 
I have encoded "Missing" with value -3. Then after adding 1 day "Missing" will be all observations with value -2.'''

target = {'Duration': {'Missing': -3}}
barmer = barmer.replace(target)

In [375]:
barmer['Duration'] = barmer['Duration'].astype(str).astype(int)
barmer['Duration'].dtypes # Returns int type

dtype('int64')

In [376]:
# Adding 1 day to all observations
barmer['Duration'] = barmer['Duration'] + 1

In [377]:
# Checking that it worked
barmer.head()

,Unnamed: 0,Lnd,Datum Beginn,Datum Ende,Wirtschaftssektor,Duration
0,0,FR,2019-07-02,2019-07-04 00:00:00,21,3
1,1,AT,2019-06-12,2019-06-12 00:00:00,21,1
2,2,IT,2019-06-16,2019-06-20 00:00:00,21,5
3,3,PL,2019-07-08,2019-07-11 00:00:00,21,4
4,4,BE,2019-06-26,2019-06-27 00:00:00,21,2


In [378]:
# Encoding -2 back to "Missing"
encode_back = {'Duration': {'-2': 'Missing'}}
barmer['Duration'] = barmer['Duration'].astype(str)
barmer = barmer.replace(encode_back)
barmer.head()

,Unnamed: 0,Lnd,Datum Beginn,Datum Ende,Wirtschaftssektor,Duration
0,0,FR,2019-07-02,2019-07-04 00:00:00,21,3
1,1,AT,2019-06-12,2019-06-12 00:00:00,21,1
2,2,IT,2019-06-16,2019-06-20 00:00:00,21,5
3,3,PL,2019-07-08,2019-07-11 00:00:00,21,4
4,4,BE,2019-06-26,2019-06-27 00:00:00,21,2


Checking if my encoding counts match expectations -> So not observation lost somewhere

In [379]:
# Missing observations like before? -> YES
(barmer['Duration']=='Missing').sum()

32

In [380]:
# Not encoded back observations? -> NO, so no bugs
(barmer['Duration']=='-3').sum()

0

In [381]:
# Not encoded back observations? -> NO, so no bugs
(barmer['Duration']=='-2').sum()

0

Data looks clean will remove the first column for aesthetics and Barmer is ready to be used for tasks!

In [382]:
barmer = barmer.drop(['Unnamed: 0'], axis = 1)
barmer.head()

,Lnd,Datum Beginn,Datum Ende,Wirtschaftssektor,Duration
0,FR,2019-07-02,2019-07-04 00:00:00,21,3
1,AT,2019-06-12,2019-06-12 00:00:00,21,1
2,IT,2019-06-16,2019-06-20 00:00:00,21,5
3,PL,2019-07-08,2019-07-11 00:00:00,21,4
4,BE,2019-06-26,2019-06-27 00:00:00,21,2


## 4. Translating everything into English

In [383]:
barmer = barmer.rename(columns = {'Lnd': 'Land', 'Datum Beginn': 'Start Date', 'Datum Ende': 'End Date', 'Wirtschaftssektor': 'Economic Sector', 'Duration': 'Duration'})
barmer.head()

,Land,Start Date,End Date,Economic Sector,Duration
0,FR,2019-07-02,2019-07-04 00:00:00,21,3
1,AT,2019-06-12,2019-06-12 00:00:00,21,1
2,IT,2019-06-16,2019-06-20 00:00:00,21,5
3,PL,2019-07-08,2019-07-11 00:00:00,21,4
4,BE,2019-06-26,2019-06-27 00:00:00,21,2


## 5. Recreating 1st bundle of Tasks - Barmer only: With imporvements and in English


__Reason for Barmer only:__ Yvonne wants to have detailed data for Barmer seperately.

#### [TASK 1]: Total amount by destination

In [384]:
amount_by_destination = barmer['Land'].value_counts()
# amount_by_destination.shape # 33 Match
# amount_by_destination.sum() # All 240410 observations included

In [385]:
barmer_task1 = amount_by_destination.to_frame()
barmer_task1 = barmer_task1.rename_axis('Destination', axis=0)
barmer_task1 = barmer_task1.rename(columns={'Land': 'Total amount'})
barmer_task1 = barmer_task1.reset_index()

# Checking results: 
# barmer_task1.shape # (33,2) dimensions
# barmer_task1.sum() # 240410 observations

# Saving data into
barmer_task1.to_excel('Barmer/BarmerTaskData/EnglishTotalAmountByDestination.xlsx')
barmer_task1.head()

,Destination,Total amount
0,AT,38661
1,CH,26530
2,FR,26105
3,NL,23448
4,IT,15998


#### [TASK 2]: Total amount by sector

In [386]:
amount_by_sector = barmer['Economic Sector'].value_counts()
# amount_by_sector.shape # 24 categories
# amount_by_sector.sum() # All 240410 observations included

In [387]:
# Disctionary with enconding variables for sector
english_sectors = {"Economic Sector":  

           {1: "Agriculture, forestry and fishing", 
            2: "Mining and quarrying", 
            3: "Manufacturing", 4: "Energy supply",
            5: "Water supply; Sewerage, waste management and remediation activities", 
            6: "Construction", 7: "Retail and wholesale",
            8: "Transportation (except carriage of goods by road) and storage", 
            9: "Transportation (considering carriage of goods by road)", 
            10: "Hospitality / Accommodation and gastronomy",
            11: "Information and communication", 12: "Provision of financial and insurance services", 
            13: "Housing",
            14: "Provision of professional, scientific and technical activities", 
            15: "Provision of other scientific activities (except Mediation and Hiring of workers)", 
            16: "Mediation and hiring of workers",
            17: "Public administration, defence and social insurance", 
            18: "Education", 19: "Human health and social work activities",
            20: "Arts, Entertainment and Recreation activities", 21: "Other service activities", 
            22: "Private households"}}

In [388]:
barmer_task2 = amount_by_sector.to_frame()
barmer_task2 = barmer_task2.rename_axis('Economic Sector', axis=0)
barmer_task2 = barmer_task2.rename(columns={'Economic Sector': 'Total amount'})
barmer_task2 = barmer_task2.reset_index()
barmer_task2 = barmer_task2.replace(english_sectors)

# Saving into 
barmer_task2.to_excel('Barmer/BarmerTaskData/EnglishTotalAmountBySector.xlsx')
barmer_task2.head(30) # # Adding 0 Category into "Missing" Manually in excel sheets!

,Economic Sector,Total amount
0,Manufacturing,91272
1,Other service activities,31706
2,Retail and wholesale,19768
3,Provision of other scientific activities (exce...,16023
4,"Provision of professional, scientific and tech...",13990
5,Construction,10133
6,Information and communication,9960
7,"Public administration, defence and social insu...",7480
8,Transportation (except carriage of goods by ro...,5287
9,Transportation (considering carriage of goods ...,5200


#### [TASK 3]: Total amount by duration

In [389]:
amount_by_duration = barmer['Duration'].value_counts()

# amount_by_duration.shape # 608 categories
# amount_by_duration.sum() # Àll 240410 observations included!

In [390]:
barmer_task3 = amount_by_duration.to_frame()
barmer_task3 = barmer_task3.rename_axis('Duration (days)', axis=0)
barmer_task3 = barmer_task3.rename(columns={'Duration': 'Total amount'})
barmer_task3 = barmer_task3.reset_index()

barmer_task3.to_excel('Barmer/BarmerTaskData/EnglishTotalAmountByDuration.xlsx')
barmer_task3.head()

,Duration (days),Total amount
0,2,45001
1,3,41410
2,1,32488
3,5,27098
4,4,26390


#### [TASK 4]: Duration by Sector

In [391]:
barmer = barmer.replace(english_sectors)
barmer.head()

,Land,Start Date,End Date,Economic Sector,Duration
0,FR,2019-07-02,2019-07-04 00:00:00,Other service activities,3
1,AT,2019-06-12,2019-06-12 00:00:00,Other service activities,1
2,IT,2019-06-16,2019-06-20 00:00:00,Other service activities,5
3,PL,2019-07-08,2019-07-11 00:00:00,Other service activities,4
4,BE,2019-06-26,2019-06-27 00:00:00,Other service activities,2


In [392]:
barmer_task4 = barmer[['Economic Sector', 'Duration']]
barmer_task4.isna().sum() # All cells have some value
barmer_task4['Economic Sector'].value_counts()

Manufacturing                                                                        91272
Other service activities                                                             31706
Retail and wholesale                                                                 19768
Provision of other scientific activities (except Mediation and Hiring of workers)    16023
Provision of professional, scientific and technical activities                       13990
Construction                                                                         10133
Information and communication                                                         9960
Public administration, defence and social insurance                                   7480
Transportation (except carriage of goods by road) and storage                         5287
Transportation (considering carriage of goods by road)                                5200
Arts, Entertainment and Recreation activities                                         4999

In [393]:
# Cleaning the 0.0 category and making it "missing"
cleanup = {'Economic Sector': {0.0: 'Missing'}}
barmer_task4 = barmer_task4.replace(cleanup)
barmer_task4['Economic Sector'].value_counts() # -> Succesful

Manufacturing                                                                        91272
Other service activities                                                             31706
Retail and wholesale                                                                 19768
Provision of other scientific activities (except Mediation and Hiring of workers)    16023
Provision of professional, scientific and technical activities                       13990
Construction                                                                         10133
Information and communication                                                         9960
Public administration, defence and social insurance                                   7480
Transportation (except carriage of goods by road) and storage                         5287
Transportation (considering carriage of goods by road)                                5200
Arts, Entertainment and Recreation activities                                         4999

In [394]:
# Transforming to numeric type for aggregation
barmer_task4['Duration'] = pd.to_numeric(barmer_task4.Duration, errors='coerce')
barmer_task4.dtypes

Economic Sector     object
Duration           float64
dtype: object

In [395]:
gpr = barmer_task4.groupby('Economic Sector')
mean_grp_sector = gpr.mean()
mean_grp_sector = mean_grp_sector.reset_index()
mean_grp_sector = mean_grp_sector.rename(columns={'Duration': 'Average Duration (days)'})
mean_grp_sector.to_excel('Barmer/BarmerTaskData/EnglishDurationBySector.xlsx')
mean_grp_sector

,Economic Sector,Average Duration (days)
0,"Agriculture, forestry and fishing",37.034106
1,"Arts, Entertainment and Recreation activities",12.142257
2,Construction,53.416839
3,Education,22.467765
4,Energy supply,29.885906
5,Hospitality / Accommodation and gastronomy,29.375171
6,Housing,38.188088
7,Human health and social work activities,21.002621
8,Information and communication,14.535542
9,Manufacturing,15.630341


#### Testing
Here I will make a short manual test for one of the sectors. If my manual calculation works then I will consider the operation to be successful!

What I am doing is the following:
1. Call all observation in one sector: (Private Households)
2. Take the mean of the numeric values: Column duration

In [396]:
test = barmer_task4[barmer_task4['Economic Sector'] == 'Private households']
test

,Economic Sector,Duration
37075,Private households,4.0
37086,Private households,2.0
37090,Private households,306.0
37201,Private households,3.0
37247,Private households,3.0
...,...,...
192314,Private households,1.0
202597,Private households,2.0
202829,Private households,6.0
202842,Private households,2.0


In [397]:
test.mean() # -> result matches

Duration    31.769231
dtype: float64

#### [TASK 5]: Duration by destination

In [398]:
barmer_task5 = barmer[['Land', 'Duration']]
barmer_task5.isna().sum() # All cells have some value

Land        0
Duration    0
dtype: int64

In [399]:
# Transforming to numeric type for aggregation
barmer_task5['Duration'] = pd.to_numeric(barmer_task5.Duration, errors='coerce')
barmer_task5.dtypes

<ipython-input-399-1cace3f658f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  barmer_task5['Duration'] = pd.to_numeric(barmer_task5.Duration, errors='coerce')


Land         object
Duration    float64
dtype: object

In [400]:
duration_by_destination = barmer_task5.groupby('Land')
mean_grp_destination = duration_by_destination.mean()
mean_grp_destination = mean_grp_destination.reset_index()
mean_grp_destination = mean_grp_destination.rename(columns={'Duration': 'Average Duration (days)'})
mean_grp_destination.to_excel('Barmer/BarmerTaskData/EnglishDurationByDestination.xlsx')
mean_grp_destination

,Land,Average Duration (days)
0,AT,26.693392
1,BE,23.104288
2,BG,18.209144
3,CH,19.335193
4,CY,29.962264
5,CZ,21.048564
6,DE,20.616162
7,DK,21.338023
8,EE,14.210390
9,ES,25.774236


#### [TASK 6 and 7]: Over time (monthly) total numbers && Sector hit the hardest in period

In [401]:
# Preparing: Using starting date and every cell represents a count of 1. Total numbers in a monthly basis will be the counts per time.
barmer_task6 = barmer[['Start Date', 'Economic Sector']]
barmer_task6 = barmer_task6.replace(cleanup)
barmer_task6['Count'] = 1
barmer_task6.dtypes

Start Date         datetime64[ns]
Economic Sector            object
Count                       int64
dtype: object

In [402]:
# Filtering time frame: Setting time period. Start and End respectively.
start, end = '2019-10-01 00:00:00', '2021-03-31 00:00:00'
after_start = barmer_task6['Start Date'] >= start
before_end = barmer_task6['Start Date'] <= end
between_two_dates = after_start & before_end
filtered_frame = barmer_task6.loc[between_two_dates]
filtered_frame

,Start Date,Economic Sector,Count
11,2019-10-04,Other service activities,1
23,2019-10-08,Other service activities,1
24,2019-10-07,Other service activities,1
35,2019-10-06,Other service activities,1
52,2019-10-17,Other service activities,1
...,...,...,...
240403,2019-11-28,Other service activities,1
240404,2019-11-19,Other service activities,1
240406,2019-11-25,Other service activities,1
240407,2019-12-15,Other service activities,1


In [403]:
grp_date = filtered_frame.set_index('Start Date')
grp_date

,Economic Sector,Count
Start Date,,
2019-10-04,Other service activities,1
2019-10-08,Other service activities,1
2019-10-07,Other service activities,1
2019-10-06,Other service activities,1
2019-10-17,Other service activities,1
...,...,...
2019-11-28,Other service activities,1
2019-11-19,Other service activities,1
2019-11-25,Other service activities,1


In [404]:
# Visualizing within a month the count per days of month
monthly_and_days = grp_date.groupby([pd.Grouper(freq="1M"), 'Start Date']).sum()
monthly_and_sector = grp_date.groupby([pd.Grouper(freq="1M"), 'Economic Sector']).sum()
monthly_and_days.head()

Count
Start Date Start Date       
2019-10-31 2019-10-01    789
           2019-10-02    341
           2019-10-03    180
           2019-10-04    182
           2019-10-05    152

In [405]:
# Summarizing months
monthly_only = grp_date.groupby([pd.Grouper(freq="1M")]).sum()
monthly_only = monthly_only.rename_axis('Month')
#monthly_only.to_excel('Barmer/BarmerTaskData/EnglishTotalNumbersOverTime.xlsx')
monthly_only

,Count
Month,
2019-10-31,15693
2019-11-30,16070
2019-12-31,9552
2020-01-31,14248
2020-02-29,14963
2020-03-31,11791
2020-04-30,2533
2020-05-31,3129
2020-06-30,5779


In [406]:
monthly_and_sector

Count
Start Date Economic Sector                                          
2019-10-31 Agriculture, forestry and fishing                      41
           Arts, Entertainment and Recreation activities         359
           Construction                                          436
           Education                                             189
           Energy supply                                         191
...                                                              ...
2021-03-31 Public administration, defence and social insur...      5
           Retail and wholesale                                  230
           Transportation (considering carriage of goods b...    146
           Transportation (except carriage of goods by roa...     50
           Water supply; Sewerage, waste management and re...     25

[399 rows x 1 columns]

In [407]:
monthly_and_sector = monthly_and_sector.reset_index()
monthly_and_sector

,Start Date,Economic Sector,Count
0,2019-10-31,"Agriculture, forestry and fishing",41
1,2019-10-31,"Arts, Entertainment and Recreation activities",359
2,2019-10-31,Construction,436
3,2019-10-31,Education,189
4,2019-10-31,Energy supply,191
...,...,...,...
394,2021-03-31,"Public administration, defence and social insu...",5
395,2021-03-31,Retail and wholesale,230
396,2021-03-31,Transportation (considering carriage of goods ...,146
397,2021-03-31,Transportation (except carriage of goods by ro...,50


In [408]:
# target = ['level_0', 'index'] # Define target columns
# monthly_and_sector = monthly_and_sector.drop(target, axis = 1) Remove columns
monthly_and_sector.to_excel('Barmer/BarmerTaskData/EnglishSectorOverTimeTotalNumbers.xlsx')
monthly_and_sector

,Start Date,Economic Sector,Count
0,2019-10-31,"Agriculture, forestry and fishing",41
1,2019-10-31,"Arts, Entertainment and Recreation activities",359
2,2019-10-31,Construction,436
3,2019-10-31,Education,189
4,2019-10-31,Energy supply,191
...,...,...,...
394,2021-03-31,"Public administration, defence and social insu...",5
395,2021-03-31,Retail and wholesale,230
396,2021-03-31,Transportation (considering carriage of goods ...,146
397,2021-03-31,Transportation (except carriage of goods by ro...,50


#### [TASK 8]: Sector 14 over time

In [409]:
# Removinf all other sector besides sector 14
barmer_task8 = barmer
barmer_task8 = barmer_task8.drop(barmer_task8[barmer_task8['Economic Sector'] != 'Provision of professional, scientific and technical activities'].index)
barmer_task8 = barmer_task8.set_index('Start Date')
barmer_task8['Count'] = 1
barmer_task8

,Land,End Date,Economic Sector,Duration,Count
Start Date,,,,,
2019-06-12,AT,2019-06-12 00:00:00,"Provision of professional, scientific and tech...",1,1
2019-06-19,AT,2019-06-19 00:00:00,"Provision of professional, scientific and tech...",1,1
2019-06-14,FR,2019-06-17 00:00:00,"Provision of professional, scientific and tech...",4,1
2019-08-28,FR,2019-08-29 00:00:00,"Provision of professional, scientific and tech...",2,1
2019-05-05,AT,2019-05-06 00:00:00,"Provision of professional, scientific and tech...",2,1
...,...,...,...,...,...
2021-03-22,HU,2021-03-26 00:00:00,"Provision of professional, scientific and tech...",5,1
2021-03-23,PL,2021-03-24 00:00:00,"Provision of professional, scientific and tech...",2,1
2021-03-22,AT,2021-03-26 00:00:00,"Provision of professional, scientific and tech...",5,1


In [410]:
duration_totalAmount_overTime = barmer_task8.groupby([pd.Grouper(freq="1M"), 'Economic Sector', 'Duration']).sum()
duration_totalAmount_overTime = duration_totalAmount_overTime.reset_index()
duration_totalAmount_overTime = duration_totalAmount_overTime.rename(columns={'Duration': 'Duration (in days)', 'Count': 'Total Amount'})
duration_totalAmount_overTime.to_excel('Barmer/BarmerTaskData/EnglishSector14Duration&TotalOverTime.xlsx')
duration_totalAmount_overTime

,Start Date,Economic Sector,Duration (in days),Total Amount
0,2019-01-31,"Provision of professional, scientific and tech...",1,18
1,2019-01-31,"Provision of professional, scientific and tech...",10,5
2,2019-01-31,"Provision of professional, scientific and tech...",11,2
3,2019-01-31,"Provision of professional, scientific and tech...",12,8
4,2019-01-31,"Provision of professional, scientific and tech...",13,3
...,...,...,...,...
1232,2021-06-30,"Provision of professional, scientific and tech...",14,2
1233,2021-06-30,"Provision of professional, scientific and tech...",5,2
1234,2021-07-31,"Provision of professional, scientific and tech...",26,2
1235,2021-08-31,"Provision of professional, scientific and tech...",4,1


#### [TASK 9]: Sector 10 Total over time

In [411]:
barmer_task9 = barmer
barmer_task9 = barmer_task9.drop(barmer_task9[barmer_task9['Economic Sector'] != 'Hospitality / Accommodation and gastronomy'].index)
barmer_task9 = barmer_task9.set_index('Start Date')
barmer_task9['Count'] = 1
barmer_task9

,Land,End Date,Economic Sector,Duration,Count
Start Date,,,,,
2019-05-01,GR,2019-10-31 00:00:00,Hospitality / Accommodation and gastronomy,184,1
2019-09-04,ES,2019-09-06 00:00:00,Hospitality / Accommodation and gastronomy,3,1
2019-06-24,ES,2019-06-26 00:00:00,Hospitality / Accommodation and gastronomy,3,1
2019-03-11,BE,2019-03-15 00:00:00,Hospitality / Accommodation and gastronomy,5,1
2019-07-02,CZ,2019-07-10 00:00:00,Hospitality / Accommodation and gastronomy,9,1
...,...,...,...,...,...
2019-09-07,AT,2019-09-07 00:00:00,Hospitality / Accommodation and gastronomy,1,1
2019-06-15,BE,2019-06-16 00:00:00,Hospitality / Accommodation and gastronomy,2,1
2019-06-29,BE,2019-06-30 00:00:00,Hospitality / Accommodation and gastronomy,2,1


In [412]:
totalAmount_overTime = barmer_task9.groupby([pd.Grouper(freq="1M"), 'Economic Sector', 'Duration']).sum()
totalAmount_overTime = totalAmount_overTime.reset_index()
totalAmount_overTime = totalAmount_overTime.rename(columns={'Duration': 'Duration (in days)', 'Count': 'Total Amount'})
totalAmount_overTime.to_excel('Barmer/BarmerTaskData/EnglishSector10OverTime.xlsx')
totalAmount_overTime

,Start Date,Economic Sector,Duration (in days),Total Amount
0,2019-01-31,Hospitality / Accommodation and gastronomy,1,3
1,2019-01-31,Hospitality / Accommodation and gastronomy,1096,1
2,2019-01-31,Hospitality / Accommodation and gastronomy,120,1
3,2019-01-31,Hospitality / Accommodation and gastronomy,121,1
4,2019-01-31,Hospitality / Accommodation and gastronomy,15,1
...,...,...,...,...
453,2021-04-30,Hospitality / Accommodation and gastronomy,47,1
454,2021-04-30,Hospitality / Accommodation and gastronomy,5,1
455,2021-04-30,Hospitality / Accommodation and gastronomy,6,1
456,2021-04-30,Hospitality / Accommodation and gastronomy,8,2


# 6. 2nd bundle of Tasks - Barmer  + Techniker

## Tasks for combined data
    [x] Tranform Barmer into TK categories
    [x] 1. Total amount by destination
    [x] 2. Total amount by sector
    [x] 3. Total amount by duration (in intervals)
    [] 5. Duration (in intervals) by destination
    [] 6. Over time (for every month 10/19 – 03/21, so don’t summarize all Januaries) total numbers, skip 04/21 because it is incomplete
    [] 7. Certain sectors that seem to be hit harderst by Covid: Over time (for every month 10/19 – 03/21, so don’t summarize all Januaries) total numbers
    [] 9. Sector 10: total over time

Encoding Logic Proposed by Yvonne

    < 1 monat    = <30 days
    1-3 monate = >29 days; <90 days
    3-12 monate = >89 days; <366 days
    12-24 monate = >365 days; <731 days
    > 24 monate = >730 days

In [413]:
test = barmer
test[test['Duration'] == 'Missing'].count()

Land               32
Start Date         32
End Date           32
Economic Sector    32
Duration           32
dtype: int64

In [414]:
test = test.replace(target)
test['Duration'] = test['Duration'].astype(str).astype(int)
test['Duration'].dtypes # Returns int type
test[test['Duration'] == -3].count()

Land               32
Start Date         32
End Date           32
Economic Sector    32
Duration           32
dtype: int64

In [415]:
test['Duration Transformed'] = 'Untransformed'
test

,Land,Start Date,End Date,Economic Sector,Duration,Duration Transformed
0,FR,2019-07-02,2019-07-04 00:00:00,Other service activities,3,Untransformed
1,AT,2019-06-12,2019-06-12 00:00:00,Other service activities,1,Untransformed
2,IT,2019-06-16,2019-06-20 00:00:00,Other service activities,5,Untransformed
3,PL,2019-07-08,2019-07-11 00:00:00,Other service activities,4,Untransformed
4,BE,2019-06-26,2019-06-27 00:00:00,Other service activities,2,Untransformed
...,...,...,...,...,...,...
240405,CH,2019-01-09,2020-01-11 00:00:00,Other service activities,368,Untransformed
240406,HR,2019-11-25,2019-11-27 00:00:00,Other service activities,3,Untransformed
240407,AT,2019-12-15,2019-12-16 00:00:00,Other service activities,2,Untransformed
240408,IT,2019-09-18,2019-09-20 00:00:00,Other service activities,3,Untransformed


In [416]:
test.loc[(test['Duration'] <= -1) & (test['Duration Transformed'] == 'Untransformed'), 'Duration Transformed'] = 'Missing'
test['Duration Transformed'].value_counts()

Untransformed    240378
Missing              32
Name: Duration Transformed, dtype: int64

In [417]:
test.loc[(test['Duration'] <= 29) & (test['Duration Transformed'] == 'Untransformed'), 'Duration Transformed'] = '< 1 monat'
test['Duration Transformed'].value_counts()

< 1 monat        220311
Untransformed     20067
Missing              32
Name: Duration Transformed, dtype: int64

In [418]:
test.loc[(test['Duration'] <= 89) & (test['Duration Transformed'] == 'Untransformed'), 'Duration Transformed'] = '1-3 monate'
test['Duration Transformed'].value_counts()

< 1 monat        220311
Untransformed     10990
1-3 monate         9077
Missing              32
Name: Duration Transformed, dtype: int64

In [419]:
test.loc[(test['Duration'] <= 365) & (test['Duration Transformed'] == 'Untransformed'), 'Duration Transformed'] = '3-12 monate'
test['Duration Transformed'].value_counts()

< 1 monat        220311
1-3 monate         9077
3-12 monate        7571
Untransformed      3419
Missing              32
Name: Duration Transformed, dtype: int64

In [420]:
test.loc[(test['Duration'] <= 730) & (test['Duration Transformed'] == 'Untransformed'), 'Duration Transformed'] = '12-24 monate'
test['Duration Transformed'].value_counts()

< 1 monat        220311
1-3 monate         9077
3-12 monate        7571
12-24 monate       2180
Untransformed      1239
Missing              32
Name: Duration Transformed, dtype: int64

In [421]:
test.loc[(test['Duration'] >= 731) & (test['Duration Transformed'] == 'Untransformed'), 'Duration Transformed'] = '> 24 monate'
test['Duration Transformed'].value_counts()

< 1 monat       220311
1-3 monate        9077
3-12 monate       7571
12-24 monate      2180
> 24 monate       1239
Missing             32
Name: Duration Transformed, dtype: int64

In [422]:
test

,Land,Start Date,End Date,Economic Sector,Duration,Duration Transformed
0,FR,2019-07-02,2019-07-04 00:00:00,Other service activities,3,< 1 monat
1,AT,2019-06-12,2019-06-12 00:00:00,Other service activities,1,< 1 monat
2,IT,2019-06-16,2019-06-20 00:00:00,Other service activities,5,< 1 monat
3,PL,2019-07-08,2019-07-11 00:00:00,Other service activities,4,< 1 monat
4,BE,2019-06-26,2019-06-27 00:00:00,Other service activities,2,< 1 monat
...,...,...,...,...,...,...
240405,CH,2019-01-09,2020-01-11 00:00:00,Other service activities,368,12-24 monate
240406,HR,2019-11-25,2019-11-27 00:00:00,Other service activities,3,< 1 monat
240407,AT,2019-12-15,2019-12-16 00:00:00,Other service activities,2,< 1 monat
240408,IT,2019-09-18,2019-09-20 00:00:00,Other service activities,3,< 1 monat


In [423]:
tk_duration

,land,kennzahl,dauer,anzahl,jahr,monat
0,AT,Laenge der Entsendung,< 1 monat,1721,2019,1
1,AT,Laenge der Entsendung,1-3 monate,67,2019,1
2,AT,Laenge der Entsendung,3-12 monate,98,2019,1
3,AT,Laenge der Entsendung,12-24 monate,74,2019,1
4,AT,Laenge der Entsendung,> 24 monate,1,2019,1
...,...,...,...,...,...,...
3013,UK,Laenge der Entsendung,< 1 monat,60,2021,3
3014,UK,Laenge der Entsendung,1-3 monate,31,2021,3
3015,UK,Laenge der Entsendung,3-12 monate,18,2021,3
3016,UK,Laenge der Entsendung,12-24 monate,3,2021,3


In [424]:
tk_wirtschaftanzeige

,land,kennzahl,kategorie,anzahl,jahr,monat
0,AT,Anzahl Wirtschaftszweig,1,6,2019,1
1,AT,Anzahl Wirtschaftszweig,2,1,2019,1
2,AT,Anzahl Wirtschaftszweig,3,643,2019,1
3,AT,Anzahl Wirtschaftszweig,4,21,2019,1
4,AT,Anzahl Wirtschaftszweig,5,5,2019,1
...,...,...,...,...,...,...
11526,UK,Anzahl Wirtschaftszweig,15,16,2021,3
11527,UK,Anzahl Wirtschaftszweig,16,3,2021,3
11528,UK,Anzahl Wirtschaftszweig,19,7,2021,3
11529,UK,Anzahl Wirtschaftszweig,20,1,2021,3


__[Task 1]: Total Amount by destination combined data__

In [425]:
barmer_task1 # Data from Barmer

,Destination,Total amount
0,AT,38661
1,CH,26530
2,FR,26105
3,NL,23448
4,IT,15998
5,ES,13524
6,BE,13388
7,PL,12302
8,CZ,11140
9,GB,10730


In [426]:
tk_duration['anzahl'].sum() # I should ungroup tk and get a dataframe with 688899 observations. Or group once more. There are two ways of doing this. I will group once more.

688899

In [427]:
# Operating
grp_destination = tk_duration.groupby(['land']).sum()
grp_destination = grp_destination.drop(['jahr', 'monat'], axis = 1)
grp_destination['anzahl'].sum() # -> 688899 matches
grp_destination

,anzahl
land,
AT,94127
BE,38246
BG,3226
CH,71462
CY,618
CZ,28856
DK,15241
EE,1202
EL,4714


In [428]:
# Clearing and formating
# grp_destination = grp_destination.reset_index()
grp_destination = grp_destination.rename(columns = {'land': 'Land', 'anzahl': 'Total amount'})
# grp_destination = grp_destination.drop(['index'], axis = 1)
# grp_destination

In [429]:
grp_destination = grp_destination.rename(columns = {'Land': 'Destination', 'anzahl': 'Total amount'})
grp_destination

,Total amount
land,
AT,94127
BE,38246
BG,3226
CH,71462
CY,618
CZ,28856
DK,15241
EE,1202
EL,4714


In [430]:
# Merging Barmer with TK
frames = [barmer_task1, grp_destination]
task1_combined = pd.concat(frames, sort=True)
task1_final = task1_combined.groupby(['Destination']).sum()
task1_final = task1_final.reset_index()
task1_final

,Destination,Total amount
0,AT,38661
1,BE,13388
2,BG,1028
3,CH,26530
4,CY,212
5,CZ,11140
6,DE,99
7,DK,5971
8,EE,385
9,ES,13524


In [431]:
# Checking if there were missing observations on the Tk Data and there were none. Big relief!
tk_duration.isna().sum()

land        0
kennzahl    0
dauer       0
anzahl      0
jahr        0
monat       0
dtype: int64

In [432]:
task1_final.to_excel('Combined/EnglishCombinedDestinationTotalAmount.xlsx')

__[Task 2]: Total Amount by sector combined data__

In [433]:
barmer_task2 = barmer_task2.replace(cleanup)
barmer_task2 = barmer_task2.groupby(['Economic Sector']).sum()
barmer_task2 = barmer_task2.reset_index()
barmer_task2

,Economic Sector,Total amount
0,"Agriculture, forestry and fishing",733
1,"Arts, Entertainment and Recreation activities",4999
2,Construction,10133
3,Education,2125
4,Energy supply,3282
5,Hospitality / Accommodation and gastronomy,1458
6,Housing,319
7,Human health and social work activities,4959
8,Information and communication,9960
9,Manufacturing,91272


In [434]:
grp_econ_tk = tk_wirtschaftanzeige.groupby(['kategorie']).sum()
grp_econ_tk = grp_econ_tk.reset_index()
grp_econ_tk = grp_econ_tk.drop(['jahr', 'monat'], axis = 1)

In [435]:
grp_econ_tk = grp_econ_tk.rename(columns = {'kategorie': 'Economic Sector', 'anzahl': 'Total amount'})
grp_econ_tk = grp_econ_tk.replace(english_sectors)
grp_econ_tk

,Economic Sector,Total amount
0,"Agriculture, forestry and fishing",3104
1,Mining and quarrying,2546
2,Manufacturing,291168
3,Energy supply,11318
4,"Water supply; Sewerage, waste management and r...",1781
5,Construction,19602
6,Retail and wholesale,44627
7,Transportation (except carriage of goods by ro...,8098
8,Transportation (considering carriage of goods ...,6454
9,Hospitality / Accommodation and gastronomy,3282


In [436]:
tk_wirtschaftanzeige.isna().sum() # also for wirtschaftanzeige: no missing information found

land         0
kennzahl     0
kategorie    0
anzahl       0
jahr         0
monat        0
dtype: int64

In [437]:
#  Merging Barmer with TK
frames_econ = [barmer_task2, grp_econ_tk]
task2_combined = pd.concat(frames_econ)
task2_combined = task2_combined.groupby(['Economic Sector']).sum()
task2_final = task2_combined.reset_index()
task2_final

,Economic Sector,Total amount
0,"Agriculture, forestry and fishing",3837
1,"Arts, Entertainment and Recreation activities",13491
2,Construction,29735
3,Education,7015
4,Energy supply,14600
5,Hospitality / Accommodation and gastronomy,4740
6,Housing,1244
7,Human health and social work activities,16647
8,Information and communication,44023
9,Manufacturing,382440


In [438]:
task2_final.to_excel('Combined/EnglishCombinedEconSectorTotalAmount.xlsx')

__[Task 3]: Total Amount by duration combined data__

In [439]:
test

,Land,Start Date,End Date,Economic Sector,Duration,Duration Transformed
0,FR,2019-07-02,2019-07-04 00:00:00,Other service activities,3,< 1 monat
1,AT,2019-06-12,2019-06-12 00:00:00,Other service activities,1,< 1 monat
2,IT,2019-06-16,2019-06-20 00:00:00,Other service activities,5,< 1 monat
3,PL,2019-07-08,2019-07-11 00:00:00,Other service activities,4,< 1 monat
4,BE,2019-06-26,2019-06-27 00:00:00,Other service activities,2,< 1 monat
...,...,...,...,...,...,...
240405,CH,2019-01-09,2020-01-11 00:00:00,Other service activities,368,12-24 monate
240406,HR,2019-11-25,2019-11-27 00:00:00,Other service activities,3,< 1 monat
240407,AT,2019-12-15,2019-12-16 00:00:00,Other service activities,2,< 1 monat
240408,IT,2019-09-18,2019-09-20 00:00:00,Other service activities,3,< 1 monat


In [440]:
tk_duration

,land,kennzahl,dauer,anzahl,jahr,monat
0,AT,Laenge der Entsendung,< 1 monat,1721,2019,1
1,AT,Laenge der Entsendung,1-3 monate,67,2019,1
2,AT,Laenge der Entsendung,3-12 monate,98,2019,1
3,AT,Laenge der Entsendung,12-24 monate,74,2019,1
4,AT,Laenge der Entsendung,> 24 monate,1,2019,1
...,...,...,...,...,...,...
3013,UK,Laenge der Entsendung,< 1 monat,60,2021,3
3014,UK,Laenge der Entsendung,1-3 monate,31,2021,3
3015,UK,Laenge der Entsendung,3-12 monate,18,2021,3
3016,UK,Laenge der Entsendung,12-24 monate,3,2021,3


In [441]:
grp_duration_tk = tk_duration.groupby(['dauer']).count()
grp_duration_tk = grp_duration_tk.reset_index()
grp_duration_tk = grp_duration_tk.drop(['kennzahl', 'anzahl', 'jahr', 'monat'], axis = 1) 
grp_duration_tk = grp_duration_tk.rename(columns = {'dauer': 'Duration category', 'land': 'Total amount'})
grp_duration_tk

,Duration category,Total amount
0,1-3 monate,718
1,12-24 monate,583
2,3-12 monate,695
3,< 1 monat,802
4,> 24 monate,220


In [442]:
grp_duration_barmer = test.groupby(['Duration Transformed']).count() # Where does -96 comes from? Well, we have 32 missing observations encoded with 
grp_duration_barmer = grp_duration_barmer.reset_index()
grp_duration_barmer = grp_duration_barmer.drop(['Start Date', 'End Date', 'Economic Sector', 'Duration'], axis = 1) 
grp_duration_barmer = grp_duration_barmer.rename(columns = {'Duration Transformed': 'Duration category', 'Land': 'Total amount'})
grp_duration_barmer

,Duration category,Total amount
0,1-3 monate,9077
1,12-24 monate,2180
2,3-12 monate,7571
3,< 1 monat,220311
4,> 24 monate,1239
5,Missing,32


In [443]:
#  Merging Barmer with TK
frames_dauer = [grp_duration_barmer, grp_duration_tk]
task3_combined = pd.concat(frames_dauer)
task3_combined

,Duration category,Total amount
0,1-3 monate,9077
1,12-24 monate,2180
2,3-12 monate,7571
3,< 1 monat,220311
4,> 24 monate,1239
5,Missing,32
0,1-3 monate,718
1,12-24 monate,583
2,3-12 monate,695
3,< 1 monat,802


In [444]:
task3_combined = task3_combined.groupby(['Duration category']).sum()
task3_combined = task3_combined.reset_index()
task3_combined

,Duration category,Total amount
0,1-3 monate,9795
1,12-24 monate,2763
2,3-12 monate,8266
3,< 1 monat,221113
4,> 24 monate,1459
5,Missing,32


In [445]:
categories = { "Duration category" :
           {"1-3 monate": "1-3 Months", 
            "12-24 monate": "12-24 Months", 
            "3-12 monate": "3-12 Months", 
            "< 1 monat": "< 1 Month",
            "> 24 monate": "> 24 Months"}}
task3_final = task3_combined.replace(categories)
task3_final

task3_final.to_excel('Combined/EnglishCombinedDurationCategoryTotalAmount.xlsx')

__[Task 5]: Duration by destination__

In [446]:
mean_grp_destination # i wont be able to use this data because of its format!

,Land,Average Duration (days)
0,AT,26.693392
1,BE,23.104288
2,BG,18.209144
3,CH,19.335193
4,CY,29.962264
5,CZ,21.048564
6,DE,20.616162
7,DK,21.338023
8,EE,14.210390
9,ES,25.774236


In [447]:
grp_duration_destination = tk_duration
grp_duration_destination = grp_duration_destination.drop(['kennzahl', 'jahr', 'monat'] ,axis = 1)
grp_duration_destination

,land,dauer,anzahl
0,AT,< 1 monat,1721
1,AT,1-3 monate,67
2,AT,3-12 monate,98
3,AT,12-24 monate,74
4,AT,> 24 monate,1
...,...,...,...
3013,UK,< 1 monat,60
3014,UK,1-3 monate,31
3015,UK,3-12 monate,18
3016,UK,12-24 monate,3


In [448]:
# This is where I want to get but with all data instead of TK only
#grp_duration_destination['anzahl'] = grp_duration_destination['anzahl'].astype('int')
#average_category = grp_duration_destination.groupby(['land', 'dauer'])['anzahl'].mean().reset_index()
#average_category

In [449]:
# Processing 
test['Count'] = 1
test

,Land,Start Date,End Date,Economic Sector,Duration,Duration Transformed,Count
0,FR,2019-07-02,2019-07-04 00:00:00,Other service activities,3,< 1 monat,1
1,AT,2019-06-12,2019-06-12 00:00:00,Other service activities,1,< 1 monat,1
2,IT,2019-06-16,2019-06-20 00:00:00,Other service activities,5,< 1 monat,1
3,PL,2019-07-08,2019-07-11 00:00:00,Other service activities,4,< 1 monat,1
4,BE,2019-06-26,2019-06-27 00:00:00,Other service activities,2,< 1 monat,1
...,...,...,...,...,...,...,...
240405,CH,2019-01-09,2020-01-11 00:00:00,Other service activities,368,12-24 monate,1
240406,HR,2019-11-25,2019-11-27 00:00:00,Other service activities,3,< 1 monat,1
240407,AT,2019-12-15,2019-12-16 00:00:00,Other service activities,2,< 1 monat,1
240408,IT,2019-09-18,2019-09-20 00:00:00,Other service activities,3,< 1 monat,1


In [450]:
grp_land_duration_barmer = test.groupby(['Land', 'Duration Transformed'])['Count'].sum().reset_index()
grp_land_duration_barmer = grp_land_duration_barmer.rename(columns = {'Land': 'land', 'Duration Transformed': 'dauer', 'Count': 'anzahl'})
grp_land_duration_barmer

,land,dauer,anzahl
0,AT,1-3 monate,1388
1,AT,12-24 monate,483
2,AT,3-12 monate,1500
3,AT,< 1 monat,34999
4,AT,> 24 monate,282
...,...,...,...
171,SK,1-3 monate,103
172,SK,12-24 monate,16
173,SK,3-12 monate,35
174,SK,< 1 monat,3074


In [451]:
# Merging
frames_av_dauer = [grp_duration_destination, grp_land_duration_barmer]
task5_combined = pd.concat(frames_av_dauer)
task5_combined

,land,dauer,anzahl
0,AT,< 1 monat,1721
1,AT,1-3 monate,67
2,AT,3-12 monate,98
3,AT,12-24 monate,74
4,AT,> 24 monate,1
...,...,...,...
171,SK,1-3 monate,103
172,SK,12-24 monate,16
173,SK,3-12 monate,35
174,SK,< 1 monat,3074


In [452]:
task5_combined = task5_combined.groupby(['land','dauer'])['anzahl'].sum()
task5_combined = task5_combined.reset_index()
task5_combined

,land,dauer,anzahl
0,AT,1-3 monate,4215
1,AT,12-24 monate,1805
2,AT,3-12 monate,4752
3,AT,< 1 monat,121602
4,AT,> 24 monate,405
...,...,...,...
181,UK,1-3 monate,1487
182,UK,12-24 monate,244
183,UK,3-12 monate,1115
184,UK,< 1 monat,35325


In [453]:
#combined_av_category['anzahl'] = combined_av_category['anzahl'].astype('int')
testing = task5_combined.groupby(['land', 'dauer'])['anzahl'].mean().reset_index()
testing

,land,dauer,anzahl
0,AT,1-3 monate,4215
1,AT,12-24 monate,1805
2,AT,3-12 monate,4752
3,AT,< 1 monat,121602
4,AT,> 24 monate,405
...,...,...,...
181,UK,1-3 monate,1487
182,UK,12-24 monate,244
183,UK,3-12 monate,1115
184,UK,< 1 monat,35325


In [454]:
task5_combined.to_excel('Combined/EnglishCombinedDurationByDestination.xlsx')

___[Task 6 & 7]: Combined Overtime Total Amount & Sector hit the hardest___

In [455]:
monthly_only.reset_index()

,Month,Count
0,2019-10-31,15693
1,2019-11-30,16070
2,2019-12-31,9552
3,2020-01-31,14248
4,2020-02-29,14963
5,2020-03-31,11791
6,2020-04-30,2533
7,2020-05-31,3129
8,2020-06-30,5779
9,2020-07-31,6732


In [472]:
tk_duration
copy10 = tk_duration

In [457]:
tk_wirtschaftanzeige

,land,kennzahl,kategorie,anzahl,jahr,monat
0,AT,Anzahl Wirtschaftszweig,1,6,2019,1
1,AT,Anzahl Wirtschaftszweig,2,1,2019,1
2,AT,Anzahl Wirtschaftszweig,3,643,2019,1
3,AT,Anzahl Wirtschaftszweig,4,21,2019,1
4,AT,Anzahl Wirtschaftszweig,5,5,2019,1
...,...,...,...,...,...,...
11526,UK,Anzahl Wirtschaftszweig,15,16,2021,3
11527,UK,Anzahl Wirtschaftszweig,16,3,2021,3
11528,UK,Anzahl Wirtschaftszweig,19,7,2021,3
11529,UK,Anzahl Wirtschaftszweig,20,1,2021,3


In [458]:
tk_wirtschaftanzeige['Day'] = 1 
tk_wirtschaftanzeige = tk_wirtschaftanzeige.rename(columns={'jahr': 'Year', 'monat': 'Month'})
tk_wirtschaftanzeige['Start Date'] = pd.to_datetime(tk_wirtschaftanzeige[['Year', 'Month', 'Day']])
tk_wirtschaftanzeige = tk_wirtschaftanzeige.drop(['kennzahl', 'Year', 'Month', 'Day'], axis = 1)

In [459]:
tk_wirtschaftanzeige = tk_wirtschaftanzeige.rename(columns = {'land': 'Land', 'kategorie': 'Economic Sector', 'anzahl': 'Total amount'})
tk_wirtschaftanzeige = tk_wirtschaftanzeige.replace(english_sectors)
tk_wirtschaftanzeige = tk_wirtschaftanzeige.drop(['Land'], axis = 1)

In [460]:
copy6 = tk_wirtschaftanzeige 
copy7 = tk_wirtschaftanzeige
copy6.dtypes

Economic Sector            object
Total amount                int64
Start Date         datetime64[ns]
dtype: object

In [461]:
copy6_grp = copy6.set_index('Start Date')
copy6_grp

,Economic Sector,Total amount
Start Date,,
2019-01-01,"Agriculture, forestry and fishing",6
2019-01-01,Mining and quarrying,1
2019-01-01,Manufacturing,643
2019-01-01,Energy supply,21
2019-01-01,"Water supply; Sewerage, waste management and r...",5
...,...,...
2021-03-01,Provision of other scientific activities (exce...,16
2021-03-01,Mediation and hiring of workers,3
2021-03-01,Human health and social work activities,7


In [462]:
monthly_amount_tk = copy6_grp.groupby([pd.Grouper(freq="1M")]).sum().reset_index()
monthly_amount_tk

,Start Date,Total amount
0,2019-01-31,11627
1,2019-02-28,18317
2,2019-03-31,24826
3,2019-04-30,24024
4,2019-05-31,35624
5,2019-06-30,32602
6,2019-07-31,38948
7,2019-08-31,47911
8,2019-09-30,58598
9,2019-10-31,57087


In [463]:
monthly_only = monthly_only.reset_index()
monthly_only

,Month,Count
0,2019-10-31,15693
1,2019-11-30,16070
2,2019-12-31,9552
3,2020-01-31,14248
4,2020-02-29,14963
5,2020-03-31,11791
6,2020-04-30,2533
7,2020-05-31,3129
8,2020-06-30,5779
9,2020-07-31,6732


In [464]:
# Merging
monthly_amount_tk = monthly_amount_tk.rename(columns = {'Start Date': 'Month', 'Total amount': 'Count'})
overtime_frames = [monthly_amount_tk , monthly_only] 
task6_combined = pd.concat(overtime_frames)
task6_combined

,Month,Count
0,2019-01-31,11627
1,2019-02-28,18317
2,2019-03-31,24826
3,2019-04-30,24024
4,2019-05-31,35624
5,2019-06-30,32602
6,2019-07-31,38948
7,2019-08-31,47911
8,2019-09-30,58598
9,2019-10-31,57087


In [465]:
task6_combined = task6_combined.groupby(['Month'])['Count'].sum()
task6_combined = task6_combined.reset_index()
task6_combined = task6_combined.rename(columns = {'Count': 'Total amount'})
task6_combined

,Month,Total amount
0,2019-01-31,11627
1,2019-02-28,18317
2,2019-03-31,24826
3,2019-04-30,24024
4,2019-05-31,35624
5,2019-06-30,32602
6,2019-07-31,38948
7,2019-08-31,47911
8,2019-09-30,58598
9,2019-10-31,72780


In [466]:
task6_combined.to_excel('Combined/EnglishCombinedTotalAmountOverTime.xlsx')

In [467]:
copy7 = copy7.set_index('Start Date')
copy7 = copy7.groupby([pd.Grouper(freq="1M"), 'Economic Sector']).sum()

In [468]:
copy7 = copy7.reset_index()
copy7 = copy7.rename(columns = {'Total amount': 'Count'})
copy7

,Start Date,Economic Sector,Count
0,2019-01-31,"Agriculture, forestry and fishing",36
1,2019-01-31,"Arts, Entertainment and Recreation activities",171
2,2019-01-31,Construction,355
3,2019-01-31,Education,73
4,2019-01-31,Energy supply,135
...,...,...,...
565,2021-03-31,"Public administration, defence and social insu...",15
566,2021-03-31,Retail and wholesale,569
567,2021-03-31,Transportation (considering carriage of goods ...,126
568,2021-03-31,Transportation (except carriage of goods by ro...,114


In [469]:
sector_monthly_frames = [copy7, monthly_and_sector]
task7_combined = pd.concat(sector_monthly_frames)
task7_combined

,Start Date,Economic Sector,Count
0,2019-01-31,"Agriculture, forestry and fishing",36
1,2019-01-31,"Arts, Entertainment and Recreation activities",171
2,2019-01-31,Construction,355
3,2019-01-31,Education,73
4,2019-01-31,Energy supply,135
...,...,...,...
394,2021-03-31,"Public administration, defence and social insu...",5
395,2021-03-31,Retail and wholesale,230
396,2021-03-31,Transportation (considering carriage of goods ...,146
397,2021-03-31,Transportation (except carriage of goods by ro...,50


In [471]:
task7_combined = task7_combined.groupby(['Start Date', 'Economic Sector']).sum()
task7_combined = task7_combined.reset_index()
#task7_combined = task7_combined.rename(columns = {'Count': 'Total amount', 'Start Date': 'Month'})
task7_combined.to_excel('Combined/EnglishCombinedSectorTotalAmountOverTime.xlsx')

__[Task 9]: Sector 10 Combined Over Time__

In [475]:
task10_combined = task7_combined.drop(task7_combined[task7_combined['Economic Sector'] != 'Hospitality / Accommodation and gastronomy'].index)
task10_combined.to_excel('Combined/EnglishCombinedSector10.xlsx')